In [6]:
import json
import time
import numpy as np

In [4]:
f = open("../files/formations.json")
d = json.load(f)

In [5]:
print(d["formations"][0]["positions_list"][0])

{'ID': 0, 'position': 'GK', 'connections_list': [1, 2]}


In [14]:
n_pos = len(d["formations"][0]["positions_list"])
link_matrix = np.zeros((n_pos,n_pos), dtype=int)
for i, pos in enumerate(d["formations"][0]["positions_list"]):
    link_matrix[i, pos["connections_list"]] = 1       

In [20]:
print(link_matrix)

[[0 1 1 0 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 0 0 0]
 [1 1 0 0 0 1 1 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 1 1 1 0 0]
 [0 0 1 0 0 0 0 0 0 1 0]
 [0 0 1 0 1 0 0 0 0 1 1]
 [0 0 0 1 1 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 1 1 0 0 0 1]
 [0 0 0 0 0 0 1 0 1 1 0]]
